# 🏠 Ev Fiyat Tahmin Projesi - Veri Keşfi ve Analizi

Bu notebook, Kaggle House Prices veri seti üzerinde veri keşfi ve temel analiz adımlarını içerir.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: f"{x:.3f}")

print("✅ Kütüphaneler yüklendi ve ayarlandı")


## 📥 Veri Yükleme


In [ ]:
train = pd.read_csv('datasets/housePrice/house_price_train.csv')
test = pd.read_csv('datasets/housePrice/house_price_test.csv')
print(f"Train şekli: {train.shape}")
print(f"Test şekli : {test.shape}")
train.head()


## 🔎 Hızlı Genel Bakış

Aşağıda sayısal sütunların temel istatistikleri ve eksik değer sayıları yer alır.


In [ ]:
train.describe().T


In [ ]:
train.isnull().sum().sort_values(ascending=False).head(15)


## 🧭 Kategorik ve Sayısal Sütunlar


In [ ]:
cat_cols = train.select_dtypes(include=['object']).columns.tolist()
num_cols = train.select_dtypes(include=[np.number]).columns.tolist()
print(f"Kategorik: {len(cat_cols)} | Sayısal: {len(num_cols)}")
print("İlk 10 kategorik:", cat_cols[:10])
print("İlk 10 sayısal:  ", num_cols[:10])


## 🎯 Hedef Değişken Dağılımı


In [ ]:
import os
os.makedirs('reports/figures', exist_ok=True)

fig, axes = plt.subplots(1, 2, figsize=(12,4))
axes[0].hist(train['SalePrice'], bins=50, color='#6baed6', edgecolor='white')
axes[0].set_title('SalePrice')
axes[1].hist(np.log1p(train['SalePrice']), bins=50, color='#31a354', edgecolor='white')
axes[1].set_title('log1p(SalePrice)')
plt.tight_layout()
plt.savefig('reports/figures/target_distribution.png', dpi=150)
plt.show()


## 🔥 Korelasyon Isı Haritası (Sayısal Değişkenler)


In [ ]:
corr = train.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(12,10))
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.title('Korelasyon Matrisi')
plt.tight_layout()
plt.savefig('reports/figures/correlation_heatmap.png', dpi=150)
plt.show()

print('SalePrice ile en yüksek korelasyonlu ilk 10 değişken:')
print(corr['SalePrice'].sort_values(ascending=False).head(10))


## 🧱 Kategorik Değişkenlerin Hedefe Göre Ortalaması


In [ ]:
for col in ['MSZoning','Neighborhood','HouseStyle']:
    if col in train.columns:
        order = train.groupby(col)['SalePrice'].mean().sort_values(ascending=False).index
        plt.figure(figsize=(10,4))
        sns.barplot(x=col, y='SalePrice', data=train, order=order, palette='Blues_d')
        plt.title(f'{col} - Ortalama SalePrice')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(f'reports/figures/{col}_target_mean.png', dpi=150)
        plt.show()


## 🧩 Eksik Değer Görselleştirme


In [ ]:
na_counts = train.isnull().sum()
na_counts = na_counts[na_counts>0].sort_values(ascending=False)
plt.figure(figsize=(10,6))
na_counts.head(20).plot(kind='bar', color='#fb6a4a')
plt.title('Eksik Değer Sayıları (İlk 20)')
plt.ylabel('Adet')
plt.tight_layout()
plt.savefig('reports/figures/missing_values.png', dpi=150)
plt.show()
na_counts.head(20)


## 📈 Önemli Sayısal Değişkenler vs Hedef


In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.regplot(x='GrLivArea', y='SalePrice', data=train, scatter_kws={'alpha':0.4}, line_kws={'color':'red'})
plt.title('GrLivArea vs SalePrice')

plt.subplot(1,2,2)
if 'OverallQual' in train.columns:
    sns.boxplot(x='OverallQual', y='SalePrice', data=train, palette='viridis')
    plt.title('OverallQual vs SalePrice')
plt.tight_layout()
plt.savefig('reports/figures/key_numeric_vs_target.png', dpi=150)
plt.show()
